# 시카고 샌드위치 맛집 사이트에 접근하기!

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [2]:
url_base = "http://www.chicagomag.com"
url_sub = '/Chicago-Magazine/November-2012/Best-Sandwiches-Chicago/'
url = url_base + url_sub

html = urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [3]:
rest_list = soup.find_all(class_= 'sammy')
rest_list[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br>
Old Oak Tap<br>
<em>Read more</em> </br></br></a></div>
</div>

In [4]:
len(rest_list)

50

# 2.접근한웹페이지에서 원하는 데이터 추출하고 정리하기

In [5]:
tmp_one=rest_list[0]

In [ ]:
tmp_one=rest_list[0]

In [ ]:
tmp_one.find(class_ = 'sammy').get_text()

In [8]:
tmp_one.find(class_ = 'sammyListing').get_text()

'BLT\r\nOld Oak Tap\nRead more '

In [9]:
tmp_one.find('a')['href']

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [10]:
tmp_str = tmp_one.find(class_='sammyListing').get_text()
tmp_str.split('\n')


['BLT\r', 'Old Oak Tap', 'Read more ']

In [11]:

tmp_res = tmp_str.split('\n')
menu = tmp_res[0].replace('\r', '')
menu

'BLT'

In [12]:
cafe = tmp_res[1]
cafe

'Old Oak Tap'

In [13]:
import re

In [14]:
from urllib.parse import urljoin

In [15]:
rank = []
main_menu = []
cafe_name = []
url_add = []

for rest in rest_list:
    rank.append(int(rest.find(class_="sammyRank").get_text()))
    tmp_str = rest.find(class_="sammyListing").get_text()
    main_menu.append(re.split(('\n|\r\n'), tmp_str)[0])
    cafe_name.append(re.split(('\n|\r\n'), tmp_str)[1])
    url_add.append(urljoin(url_base, rest.find('a')['href']))

In [16]:
print(re.split((\n:\r\n), tmp_str)[0])
print(re.split((\n:\r\n), tmp_str)[1])

SyntaxError: unexpected character after line continuation character (<ipython-input-16-0fde21949a8f>, line 1)

In [17]:
rank[:5]

[1, 2, 3, 4, 5]

In [18]:

main_menu[:5]

['BLT', 'Fried Bologna', 'Woodland Mushroom', 'Roast Beef', 'PB&L']

In [19]:
cafe_name[:5]

['Old Oak Tap', 'Au Cheval', 'Xoco', 'Al’s Deli', 'Publican Quality Meats']

In [20]:

url_add[-5:]

['http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Chickpea-Kufta/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-The-Goddess-and-Grocer-Debbies-Egg-Salad/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Zenwich-Beef-Curry/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Toni-Patisserie-Le-Vegetarien/',
 'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Phoebes-Bakery-The-Gatsby/']

In [21]:
import pandas as pd
df = pd.DataFrame({'rank': rank, 'menu': main_menu, 'cafe': cafe_name, 'URL': url_add})
df.head()

,rank,menu,cafe,URL
0,1,BLT,Old Oak Tap,http://www.chicagomag.com/Chicago-Magazine/Nov...
1,2,Fried Bologna,Au Cheval,http://www.chicagomag.com/Chicago-Magazine/Nov...
2,3,Woodland Mushroom,Xoco,http://www.chicagomag.com/Chicago-Magazine/Nov...
3,4,Roast Beef,Al’s Deli,http://www.chicagomag.com/Chicago-Magazine/Nov...
4,5,PB&L,Publican Quality Meats,http://www.chicagomag.com/Chicago-Magazine/Nov...


In [22]:
df.to_csv('chicago1.csv', sep = ',', encoding = "UTF-8")

In [23]:
df['URL'][0]

'http://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

In [24]:
html = urlopen(df['URL'][0])
soup_tmp = BeautifulSoup(html, 'html.parser')

In [25]:
soup_tmp.find(class_='addy')

<p class="addy">
<em>$10. 2109 W. Chicago Ave., 773-772-0406, <a href="http://www.theoldoaktap.com/">theoldoaktap.com</a></em></p>

In [26]:
tmp_str = soup_tmp.find(class_='addy').get_text()
tmp_str

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

In [27]:
tmp_str.split()

['$10.', '2109', 'W.', 'Chicago', 'Ave.,', '773-772-0406,', 'theoldoaktap.com']

In [28]:
tmp_price = tmp_str.split()[0][:-1]
tmp_price


'$10'

In [29]:
tmp_str.split()[1:-2]

['2109', 'W.', 'Chicago', 'Ave.,']

In [30]:
' '.join(tmp_str.split()[1:-2])

'2109 W. Chicago Ave.,'

In [31]:
tmp_addr = ' '.join(tmp_str.split()[1:-2])[:-1]
tmp_addr

'2109 W. Chicago Ave.'

In [32]:
from tqdm import tqdm_notebook
import time

In [33]:
price = []
addr = []
for n in tqdm_notebook(df.index):
    html = urlopen(df['URL'][n])
    soup_tmp = BeautifulSoup(html, 'html.parser')
    tmp_str = soup_tmp.find(class_='addy').get_text()
    
    price.append(tmp_str.split()[0][:-1])
    addr.append(' '.join(tmp_str.split()[1:-2])[:-1])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
price[:10]

['$10', '$9', '$9.50', '$9.40', '$10', '$7.25', '$16', '$10', '$9', '$17']

In [35]:
addr[:5]

['2109 W. Chicago Ave.',
 '800 W. Randolph St.',
 '445 N. Clark St.',
 '914 Noyes St., Evanston',
 '825 W. Fulton Mkt.']

In [36]:
key_fd = open('googlemapskey.txt', mode = 'r')
gmaps_key = key_fd.read(100)
key_fd.close()

In [37]:
gmaps_key[-10:]

'aqfnU_i614'

In [38]:
lat = []
lng = []

for n in tqdm_notebook(df.index):
    if df['address'][n] != 'Multuple':
        target_name = df['address'][n]+', '+'Chicago'
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get('geometry')
        lat.append(location_output['location']['lat'])
        lng.append(location_output['location']['lng'])
    else:
        lat.append(np.nan)
        lng.append(np.nan)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


KeyError: 'address'

In [ ]:
df['lat']=lat
df['lng']=lng


In [ ]:
mapping = folium.Map(location = [df['lat'].mean(), df['lng'].mean()],
                                            zoom_start=11)
folium.Marker([df['lat'].mean(), df['lng'].mean()],
                                            popup='center').add_to(mapping)
mapping

In [ ]:
mapping = folium.Map(location = [df['lat'].mean(), df['lng'].mean()],
                                            zoom_start=11)

for n in df.index:
    if df['Address'][n]!='Multiple':
        folium.Marker([df['lat'][n], df['lng'][n]],
                                             popup = df['Cafe'][n]).add_to(mapping)

mapping

In [ ]:
import folium

In [ ]:
a=folium.Map(location=[45.5236,-122.750])
a